In [1]:
setwd("~/IBM_Attrition_DataChallenge/") # sets proper working directory

In [2]:
# import R libraries to use later
suppressPackageStartupMessages(library('caret'))
suppressPackageStartupMessages(library('mlbench'))
suppressPackageStartupMessages(library(plyr))

# Read in the Data

First read in the raw data. Then as a standard practice, return the first few rows to check the dataframe is read in correctly.

In [3]:
dataframe <- read.csv(file='WA_Fn-UseC_-HR-Employee-Attrition.csv') # IBM data is stored in variable 'dataframe'
head(dataframe)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,⋯,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,⋯,1,80,0,8,0,1,6,4,0,5
2,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,⋯,4,80,1,10,3,3,10,7,1,7
3,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,⋯,2,80,0,7,3,3,0,0,0,0
4,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,⋯,3,80,0,8,3,3,8,7,3,0
5,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,⋯,4,80,1,6,3,3,2,2,2,2
6,32,No,Travel_Frequently,1005,Research & Development,2,2,Life Sciences,1,8,⋯,3,80,0,8,2,2,7,7,3,6


# Check for Missing, Repeated, and Arbitrary Values

We will check for missing data to ensure that no rows have to be removed/edited. If some values are missing, there are multiple ways to correct the issue. We could use a deletion method (listwise, pairwise, etc.) to remove rows or variables with missing data; however, this is not best practice because it could result in losing lots of information. We could also try to fill in the data using some imputation method. Imputation methods can range from simple mean/median/mode calculations to more complex techniques like the 'nearest neighbors' method, which extracts an estimate only from rows with similar traits.

In [4]:
sum(is.na(dataframe)) # sums the number of NA (null) values in the dataframe 

[1] 0

There are no null values! This means we don't have to bother with missing data and can move on to the next step.

Next, we will remove any columns with the same value in each row, as they are the same for everyone and will provide no additional information.

In [5]:
dataframe <- dataframe[vapply(dataframe, function(x) length(unique(x)) > 1, logical(1L))] # remove cols with one value

We will now check that each employee is only entered in the dataset once. When calculating attrition, it would be fine if an employee was counted twice, say if attrition changes after some promotion/work-life change. However, when gathering general statistics on IBM employees (number of workers, distribution of jobs, etc.), we want to avoid double counting. We will do this through noticing that there is an Employee Number column with unique identifiers.

In [6]:
# check that column EmployeeNumber is the same before and after duplicate values have been removed
all(unique(dataframe$EmployeeNumber) == dataframe$EmployeeNumber) 

[1] TRUE

Great! Because the column is the same before and after duplicates are removed, there are no employee recurrences to worry about! This also means we can remove Employee Number because we no know it is arbitrary!

In [7]:
drops <- c("EmployeeNumber")
dataframe <- dataframe[ , !(names(dataframe) %in% drops)]
head(dataframe)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,⋯,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<int>,<chr>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,2,Female,⋯,3,1,0,8,0,1,6,4,0,5
2,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,3,Male,⋯,4,4,1,10,3,3,10,7,1,7
3,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,4,Male,⋯,3,2,0,7,3,3,0,0,0,0
4,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,4,Female,⋯,3,3,0,8,3,3,8,7,3,0
5,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,Male,⋯,3,4,1,6,3,3,2,2,2,2
6,32,No,Travel_Frequently,1005,Research & Development,2,2,Life Sciences,4,Male,⋯,3,3,0,8,2,2,7,7,3,6


# Remove Correlated Data

We also have to eliminate correlated data from the dataframe. Correlated data are variables that have a strong association with eachother and will thus tend to follow related patterns (note: not causation). This is an important step in cleaning the data, as we do not want multiple independent variables conveying the same information to the model. Correlated data can make models unstable and introduce variance/ovrfitting to the dataset by double-counting.

We can only check correlations between numeric values, so we will first remove the categorical variables.

In [8]:
numericData <- dataframe[sapply(dataframe, is.numeric)]  # retains only the columns following is.numeric
head(numericData)                                        # visualizes the numeric data

,Age,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,⋯,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,41,1102,1,2,2,94,3,2,4,5993,⋯,3,1,0,8,0,1,6,4,0,5
2,49,279,8,1,3,61,2,2,2,5130,⋯,4,4,1,10,3,3,10,7,1,7
3,37,1373,2,2,4,92,2,1,3,2090,⋯,3,2,0,7,3,3,0,0,0,0
4,33,1392,3,4,4,56,3,1,3,2909,⋯,3,3,0,8,3,3,8,7,3,0
5,27,591,2,1,1,40,3,1,2,3468,⋯,3,4,1,6,3,3,2,2,2,2
6,32,1005,2,2,4,79,3,1,4,3068,⋯,3,3,0,8,2,2,7,7,3,6


Now we will use the numeric data to calculate variable correlations. We will use the built-in cor function to create a matrix containing correlation values. An absolute value close to 1 represents a high correlation between variables. The cor function uses Pearson's rank-based measure of association. 

In [9]:
corrMatrix <- cor(numericData)  # generates correlation matrix from numeric data.
head(corrMatrix)                

,Age,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,⋯,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
Age,1.00000000,0.010660943,-0.001686120,0.20803373,0.01014643,0.02428654,0.029819959,0.509604228,-0.004891877,0.497854567,⋯,0.0019038955,0.053534720,0.037509712,0.680380536,-0.019620819,-0.021490028,0.311308770,0.212901056,0.21651337,0.202088602
DailyRate,0.01066094,1.000000000,-0.004985337,-0.01680643,0.01835485,0.02338142,0.046134874,0.002966335,0.030571008,0.007707059,⋯,0.0004732963,0.007846031,0.042142796,0.014514739,0.002452543,-0.037848051,-0.034054768,0.009932015,-0.03322898,-0.026363178
DistanceFromHome,-0.00168612,-0.004985337,1.000000000,0.02104183,-0.01607533,0.03113059,0.008783280,0.005302731,-0.003668839,-0.017014445,⋯,0.0271096185,0.006557475,0.044871999,0.004628426,-0.036942234,-0.026556004,0.009507720,0.018844999,0.01002884,0.014406048
Education,0.20803373,-0.016806433,0.021041826,1.00000000,-0.02712831,0.01677483,0.042437634,0.101588886,-0.011296117,0.094960677,⋯,-0.0245387912,-0.009118377,0.018422220,0.148279697,-0.025100241,0.009819189,0.069113696,0.060235554,0.05425433,0.069065378
EnvironmentSatisfaction,0.01014643,0.018354854,-0.016075327,-0.02712831,1.00000000,-0.04985696,-0.008277598,0.001211699,-0.006784353,-0.006259088,⋯,-0.0295479523,0.007665384,0.003432158,-0.002693070,-0.019359308,0.027627295,0.001457549,0.018007460,0.01619361,-0.004998723
HourlyRate,0.02428654,0.023381422,0.031130586,0.01677483,-0.04985696,1.00000000,0.042860641,-0.027853486,-0.071334624,-0.015794304,⋯,-0.0021716974,0.001330453,0.050263399,-0.002333682,-0.008547685,-0.004607234,-0.019581616,-0.024106220,-0.02671559,-0.020123200


We will now implement the findCorrelation function from the R package 'caret'. We will use this method to parse the correlation matrix, returning the index of any variables with a correlation over some value. In this case, we will use the standard 0.75 as the cutoff.

In [10]:
highCorrIndex <- findCorrelation(corrMatrix, cutoff=0.75) # returns the indexes of highly-correlated columns
corrColNames <- colnames(corrMatrix)[highCorrIndex]       # retrieves the column names from the indecies
corrColNames                                              # displays the column names to be removed

[1] "TotalWorkingYears" "YearsAtCompany"    "JobLevel"         
[4] "PercentSalaryHike"

Now that we have the high correlation columns we gotta get them outta here!

In [11]:
numericData <- numericData[!names(numericData) %in% corrColNames]  # removes columns listed in currColNames
head(numericData)

,Age,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TrainingTimesLastYear,WorkLifeBalance,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,41,1102,1,2,2,94,3,4,5993,19479,8,3,1,0,0,1,4,0,5
2,49,279,8,1,3,61,2,2,5130,24907,1,4,4,1,3,3,7,1,7
3,37,1373,2,2,4,92,2,3,2090,2396,6,3,2,0,3,3,0,0,0
4,33,1392,3,4,4,56,3,3,2909,23159,1,3,3,0,3,3,7,3,0
5,27,591,2,1,1,40,3,2,3468,16632,9,3,4,1,3,3,2,2,2
6,32,1005,2,2,4,79,3,4,3068,11864,0,3,3,0,2,2,7,3,6


# Creating Dummies for Categorical Vars and Re-Merging

Now that we have the numeric columns figured out, we will re-merge the numeric data with the categorical data. However before merging the two, we will have to save and extract the attrition column to be used for training and 
testing the model. We will alo have to convert all of the categorical variables into dummies. Dummies are created when a categorical column is converted into multiple integer columns of binary values. For example, a column of Gender with category values of Male and Female, would now become two columns in the dataframe, one of 'GenderMale' and 'GenderFemale', each with either a 1 or 0 to indicate whether the row contains that gender. This is done to allow the model to read in integers for regression and make inferences on categorical data.

We will use the fastDummies library to create the dummies.

In [12]:
library(fastDummies)
attrition <- dataframe['Attrition']                         # saves values of attrition 
dataframe <- dataframe[!names(dataframe) %in% 'Attrition']  # removes attrition, as we dont want attrition dummies
dataframe <- dummy_columns(dataframe)                       # generates and saves dummies
head(dataframe)


,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,⋯,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
,<int>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<int>,<chr>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,41,Travel_Rarely,1102,Sales,1,2,Life Sciences,2,Female,94,⋯,0,0,0,1,0,0,0,1,0,1
2,49,Travel_Frequently,279,Research & Development,8,1,Life Sciences,3,Male,61,⋯,0,0,1,0,0,0,1,0,1,0
3,37,Travel_Rarely,1373,Research & Development,2,2,Other,4,Male,92,⋯,0,0,0,0,0,0,0,1,0,1
4,33,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,4,Female,56,⋯,0,0,1,0,0,0,1,0,0,1
5,27,Travel_Rarely,591,Research & Development,2,1,Medical,1,Male,40,⋯,0,0,0,0,0,0,1,0,1,0
6,32,Travel_Frequently,1005,Research & Development,2,2,Life Sciences,4,Male,79,⋯,0,0,0,0,0,0,0,1,1,0


In [13]:
# We will now do the 're-merge' mentioned earlier, we make sure to only select the numerical dummy values from the
# dataframe, and merge those with the numerical data
df <- join(numericData, dataframe[ , purrr::map_lgl(dataframe, is.numeric)])

Joining by: Age, DailyRate, DistanceFromHome, Education, EnvironmentSatisfaction, HourlyRate, JobInvolvement, JobSatisfaction, MonthlyIncome, MonthlyRate, NumCompaniesWorked, PerformanceRating, RelationshipSatisfaction, StockOptionLevel, TrainingTimesLastYear, WorkLifeBalance, YearsInCurrentRole, YearsSinceLastPromotion, YearsWithCurrManager



In [14]:
# We finally have an all-numeric dataframe! Let's do some checks to understand our final predictive variables!
ncol(df)          
head(df)
colnames(df)

[1] 51

,Age,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobSatisfaction,MonthlyIncome,MonthlyRate,⋯,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,41,1102,1,2,2,94,3,4,5993,19479,⋯,0,0,0,1,0,0,0,1,0,1
2,49,279,8,1,3,61,2,2,5130,24907,⋯,0,0,1,0,0,0,1,0,1,0
3,37,1373,2,2,4,92,2,3,2090,2396,⋯,0,0,0,0,0,0,0,1,0,1
4,33,1392,3,4,4,56,3,3,2909,23159,⋯,0,0,1,0,0,0,1,0,0,1
5,27,591,2,1,1,40,3,2,3468,16632,⋯,0,0,0,0,0,0,1,0,1,0
6,32,1005,2,2,4,79,3,4,3068,11864,⋯,0,0,0,0,0,0,0,1,1,0


[1] "Age"                               "DailyRate"                        
 [3] "DistanceFromHome"                  "Education"                        
 [5] "EnvironmentSatisfaction"           "HourlyRate"                       
 [7] "JobInvolvement"                    "JobSatisfaction"                  
 [9] "MonthlyIncome"                     "MonthlyRate"                      
[11] "NumCompaniesWorked"                "PerformanceRating"                
[13] "RelationshipSatisfaction"          "StockOptionLevel"                 
[15] "TrainingTimesLastYear"             "WorkLifeBalance"                  
[17] "YearsInCurrentRole"                "YearsSinceLastPromotion"          
[19] "YearsWithCurrManager"              "JobLevel"                         
[21] "PercentSalaryHike"                 "TotalWorkingYears"                
[23] "YearsAtCompany"                    "BusinessTravel_Non-Travel"        
[25] "BusinessTravel_Travel_Frequently"  "BusinessTravel_Travel_Rarely"     
[27] "Department_Human Resources"        "Department_Research & Development"
[29] "Department_Sales"                  "EducationField_Human Resources"   
[31] "EducationField_Life Sciences"      "EducationField_Marketing"         
[33] "EducationField_Medical"            "EducationField_Other"             
[35] "EducationField_Technical Degree"   "Gender_Female"                    
[37] "Gender_Male"                       "JobRole_Healthcare Representative"
[39] "JobRole_Human Resources"           "JobRole_Laboratory Technician"    
[41] "JobRole_Manager"                   "JobRole_Manufacturing Director"   
[43] "JobRole_Research Director"         "JobRole_Research Scientist"       
[45] "JobRole_Sales Executive"           "JobRole_Sales Representative"     
[47] "MaritalStatus_Divorced"            "MaritalStatus_Married"            
[49] "MaritalStatus_Single"              "OverTime_No"                      
[51] "OverTime_Yes"

# Adding Attrition Back While Fixing Skew with SMOTE

For the models we will be using, we can't forget to include the attrition column! We will quickly add it to the df

In [15]:
df <- cbind(df, attrition)

Before building the model, we will want to get a count of the attrition values. This will give us a baseline value to compare our results against by giving us the result we would get by purely guessing either Yes or No. It can also be helpful in pointing out skewness

In [16]:
sum(attrition == "Yes")

[1] 237

In [17]:
sum(attrition == "No")

[1] 1233

Aha! We see that the attrition values are highly skewed. In fact, if we were to guess NO no matter what, we would be right 83% of the time! This is an important number to remember as we create models because we will want to make sure that whatever results we get are higher than 83. This will be tough though, as 83% is already pretty high. 

To improve our data even further, its good practice to remove this skew from our trainig/testing data. We will do this by using a program to create synthetic, realistic minority cases ('Yes' cases, such that the dataset will be balanced. We will do this through the SMOTE library.

In [18]:
df$Attrition <- as.factor(df$Attrition)  # this changes Attrition to a factor, a requirement of SMOTE and of later models

In [19]:
suppressPackageStartupMessages(library(DMwR))                  # library containing SMOTE function
df <- SMOTE(Attrition ~ ., df, perc.over = 600,perc.under=100) # SMOTE Function, perc used to specify number of yes/no that need to be generated

In [20]:
attrition <- df$Attrition  # we will now save our normalized attrition values

We can see now that the 'yes' rows have increased to close to the same amount as the 'no' rows, success!

In [21]:
sum(attrition == "Yes") 

[1] 1659

In [22]:
sum(attrition == "No")

[1] 1422

# Exporting the Final Dataset

In [23]:
head(df)

,Age,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobSatisfaction,MonthlyIncome,MonthlyRate,⋯,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes,Attrition
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
880,60,696,7,4,2,52,4,4,5220,10893,⋯,0,0,1,0,1,0,0,0,1,No
551,23,650,9,1,2,37,3,1,2500,4344,⋯,0,0,0,0,0,1,0,1,0,No
1056,34,829,15,3,2,71,3,1,17007,11929,⋯,1,0,0,0,1,0,0,1,0,No
489,42,622,2,4,3,81,3,4,4089,5718,⋯,0,0,0,0,0,1,0,1,0,No
173,36,1480,3,2,4,30,3,2,2088,15062,⋯,0,0,0,0,0,0,1,1,0,No
62,38,653,29,5,4,50,3,4,2406,5456,⋯,0,0,0,0,0,0,1,1,0,No


In [24]:
write.csv(df, file='attrition_model_data_cleaned.csv', row.names= FALSE)